# ENSF 592 Project: Phase II

## Data Preparation

In [1]:
import pandas as pd
import numpy as np
import folium

### Get a 10 x 10 matrix of city limits

In [2]:
# Read data for city boundaries
data_city_boundary = pd.read_csv('City_Boundary_layer.csv')

# Manipulate polygon string to get list of coordinates
polygon = data_city_boundary['the_geom'].values[0]
polygon = polygon[10:-2]
coordinates_city_limits = polygon.split(', ')

# Get a list of longitudes and latitudes for the city limits
longitudes_city_limits = []
latitudes_city_limits = []
for long_lat in coordinates_city_limits:
    long, lat = long_lat.split()
    long = float(long)
    lat = float(lat)
    longitudes_city_limits.append(long)
    latitudes_city_limits.append(lat)
    
# Find absolute city limits as the min/max longitude/latitude
overall_min_long = min(longitudes_city_limits)
overall_max_long = max(longitudes_city_limits)
overall_min_lat = min(latitudes_city_limits)
overall_max_lat = max(latitudes_city_limits)

# Split the city into a 10 x 10 matrix
# Create a 10x10 matrix
quadrant = [[0 for x in range(10)] for y in range(10)]
# Populate quadrant boundaries
for i in range(0,10):
    # Get the min/max longitude for the quadrant
    min_long = overall_min_long + (overall_max_long - overall_min_long) / 10 * i
    max_long = overall_min_long + (overall_max_long - overall_min_long) / 10 * (i + 1)
    for j in range(0, 10):
        # Get the min/max latitude for the quadrant
        min_lat = overall_min_lat + (overall_max_lat - overall_min_lat) / 10 * j
        max_lat = overall_min_lat + (overall_max_lat - overall_min_lat) / 10 * (j + 1)
        # Populate data into a dataframe for each quadrant
        quadrant[i][j] = {'min_lat':min_lat,'max_lat':max_lat,'min_long':min_long,'max_long':max_long}

### Plot a map of the city limits

In [3]:
# Start maps at the center of Calgary
map_overall_boundaries = folium.Map(location=[51.0447, -114.0719], zoom_start=10)
map_quadrant_boundaries = folium.Map(location=[51.0447, -114.0719], zoom_start=10)

# Place rectangles
# Place rectangle for the overall map
folium.vector_layers.Rectangle([(overall_max_lat, overall_max_long), \
                                (overall_max_lat, overall_min_long), \
                                (overall_min_lat, overall_max_long), \
                                (overall_min_lat, overall_min_long)]).add_to(map_overall_boundaries)
            
# Place rectangle for each quadrant
for i in range(0,10):
    for j in range(0, 10):
        folium.vector_layers.Rectangle([(quadrant[i][j]['max_lat'], quadrant[i][j]['max_long']), \
                                        (quadrant[i][j]['max_lat'], quadrant[i][j]['min_long']), \
                                        (quadrant[i][j]['min_lat'], quadrant[i][j]['max_long']), \
                                        (quadrant[i][j]['min_lat'], quadrant[i][j]['min_long'])]).add_to(map_quadrant_boundaries)

# Save maps
map_overall_boundaries.save('map_overall_boundaries.html')
map_quadrant_boundaries.save('map_quadrant_boundaries.html')

### Load and clean data

#### Speed Limit

In [4]:
# Need a dataframe with lat, long, and speed limit at the location
data_speed_limit

#### Traffic Volume

In [5]:
# Need a dataframe with lat, long, and volume at the location

#### Traffic Cameras

In [6]:
# Read csv file
data_traffic_cameras = pd.read_csv('Traffic_Camera_Locations.csv')

# Drop unwanted columns
data_traffic_cameras.drop(columns=['Camera Location', 'Quadrant', 'Camera URL'], inplace=True)

# Drop any NaN values
data_traffic_cameras.dropna(inplace=True)

# Add a count column
data_traffic_cameras['Count'] = 1

#### Traffic Signals

In [7]:
# Read csv file
data_traffic_signals = pd.read_csv('Traffic_Signals.csv')

# Drop unwanted columns
data_traffic_signals.drop(columns=['INSTDATE', 'FIRSTROAD', 'SECONDROAD', 'QUADRANT',
                                   'INT_TYPE', 'PEDBUTTONS', 'PED_TIMER', 
                                   'ACCESSIBLE PEDESTRIAN SIGNAL', 'Point'], inplace=True)

# Drop any NaN values
data_traffic_signals.dropna(inplace=True)

#### Traffic Signs

In [8]:
# Read csv file
data_traffic_signs = pd.read_csv('Traffic_Signs.csv')

# Drop unwanted columns
data_traffic_signs.drop(columns=['BLADE_TYPE', 'COMMENTS', 'FACING_CD', 'FLD_SRC_TXT', 
                                 'INSTDATE', 'MATERIAL', 'PL_TY', 'REUSE', 'SGN_STA_CD',
                                 'SIGN_TXT', 'STA_CD', 'SIZE_CD', 'SUPPORTTYPE', 'UNITID',
                                 'TE_SIGNLOCATION_UNITID', 'Ward Boundaries', 'City Quadrants',
                                 'Calgary Communities', 'Ward Boundaries 2013-2017'], inplace=True)

# Drop any NaN values
data_traffic_signs.dropna(inplace=True)

# Get long and lat from point
data_traffic_signs['longitude'] = data_traffic_signs['POINT']
data_traffic_signs['latitude'] = data_traffic_signs['POINT']
data_traffic_signs['longitude'] = data_traffic_signs['longitude'].apply(lambda x: float(x[7:x.index(' ', 7)]))
data_traffic_signs['latitude'] = data_traffic_signs['latitude'].apply(lambda x: float(x[x.index(' ', 7):-1]))

# Drop point column
data_traffic_signs.drop(columns=['POINT'], inplace=True)

# Rename count column to 'count'
data_traffic_signs.rename(columns={"SGN_COUNT_NO": "Count"}, inplace=True)

#### Weather Conditions

In [9]:
# Need a dataframe with lat, long, avg temp, and avg visibility at the location

#### Traffic Accidents

In [10]:
# Read csv file
data_traffic_accidents = pd.read_csv('Traffic_Signals.csv')

# Drop unwanted columns
data_traffic_accidents.drop(columns=['INSTDATE', 'FIRSTROAD', 'SECONDROAD', 'QUADRANT', 'INT_TYPE',
                                     'PEDBUTTONS', 'PED_TIMER', 'ACCESSIBLE PEDESTRIAN SIGNAL', 'Point'], inplace=True)

# Drop any NaN values
data_traffic_accidents.dropna(inplace=True)

### Analyse Data

In [11]:
# Add data to quadrant dictionary 
for i in range(10):
    for j in range(10):
        #quadrant[i][j]['Average Speed']
        #quadrant[i][j]['Average Traffic Volume']
        quadrant[i][j]['Number of Traffic Cameras'] = data_traffic_cameras[((data_traffic_cameras['latitude'] >= quadrant[i][j]['min_lat']) &
                                                                            (data_traffic_cameras['latitude'] < quadrant[i][j]['max_lat']) &
                                                                            (data_traffic_cameras['longitude'] >= quadrant[i][j]['min_long']) &
                                                                            (data_traffic_cameras['longitude'] < quadrant[i][j]['max_long']))]['Count'].sum()
        quadrant[i][j]['Number of Traffic Signals'] = data_traffic_signals[((data_traffic_signals['latitude'] >= quadrant[i][j]['min_lat']) &
                                                                            (data_traffic_signals['latitude'] < quadrant[i][j]['max_lat']) &
                                                                            (data_traffic_signals['longitude'] >= quadrant[i][j]['min_long']) &
                                                                            (data_traffic_signals['longitude'] < quadrant[i][j]['max_long']))]['Count'].sum()
        quadrant[i][j]['Number of Traffic Signs'] = data_traffic_signs[((data_traffic_signs['latitude'] >= quadrant[i][j]['min_lat']) &
                                                                        (data_traffic_signs['latitude'] < quadrant[i][j]['max_lat']) &
                                                                        (data_traffic_signs['longitude'] >= quadrant[i][j]['min_long']) &
                                                                        (data_traffic_signs['longitude'] < quadrant[i][j]['max_long']))]['Count'].sum()
        #quadrant[i][j]['Average Temperature']
        #quadrant[i][j]['Average Visibility']
        quadrant[i][j]['Number of Traffic Accidents'] = data_traffic_accidents[((data_traffic_accidents['latitude'] >= quadrant[i][j]['min_lat']) &
                                                                                (data_traffic_accidents['latitude'] < quadrant[i][j]['max_lat']) &
                                                                                (data_traffic_accidents['longitude'] >= quadrant[i][j]['min_long']) &
                                                                                (data_traffic_accidents['longitude'] < quadrant[i][j]['max_long']))]['Count'].sum()
        
# Visualize data

## Visualizing Speed Limits

In [47]:
# Create the map
map_speed_limits = folium.Map(location=[51.0447, -114.0719], zoom_start=10)

# Go through every road
for index, row in data_speed_limit.iterrows():
    # Get coordinates for the road
    coordinates = row['coordinates']
    # Get color intensity based on speed limit
    color = '#%02x%02x%02x' % (int(min(row['speed limit'] * 255 / 110, 255)), 0, 0)
    # Add line to the map
    folium.PolyLine(locations=coordinates,weight=5, color=color).add_to(map_speed_limits)

# Save map
map_speed_limits.save('map_speed_limits.html')

## Visualizing Traffic Heat Map

In [13]:
# Do a heat map of the traffic volume